In [1]:
import re
import json
import numpy as np
import pandas as pd
import jieba
from zhconv import convert
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

In [3]:
# 读入20220821所有数据
all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)

/var/folders/b0/f13r09ys4819g6vy91gl1_zr0000gn/T/ipykernel_13791/3330071591.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)


In [4]:
a = all_data['jobTracks']
a

0        ['{"companyName": "深圳市亿科数字科技有限公司", "descriptio...
1        ['{"companyName": "北京美餐好客科技有限公司", "companyNatu...
2        ['{"companyName": "Continental Tires (China) L...
3        ['{"companyName": "北京星制科技有限公司（全民快乐）", "descrip...
4        ['{"companyName": "酷狗音乐", "companyTags": "互联网+...
                               ...                        
81291    ['{"endDate": "2022-07", "companyName": "江门市新会...
81292    ['{"endDate": "2022-07", "companyName": "上海微创医...
81293    ['{"endDate": "2022-07", "companyName": "上海微创医...
81294    ['{"endDate": "2020-07", "companyName": "金山云",...
81295    ['{"endDate": "2022-08", "companyName": "厦门安踏服...
Name: jobTracks, Length: 81296, dtype: object

In [5]:
print(a[0])
b = json.loads(a[0])

['{"companyName": "深圳市亿科数字科技有限公司", "description": "工作职责:\\n一、公司管理\\n1、确定发展方向: 结合公司特点、竞争优势、行业机会等信息，确定公司未来主要从事的行业和赛道，如移动互联网、创意为主、技术导向三大战略；\\n2、企业文化建设: 和ceo一道，深入对公司文化进行总结、分析，通过投票、讨论、调研等方式，最终确定公司文化，并在各个事业部进行推行落实；\\n3、内部流程优化: 协同销售、运营、创意、财务、技术、商务等部门对整个公司内部的流程进行梳理，最终落脚为公司信息化系统建设，从而提升公司运转的效率和规避相关的风险；\\n4、薪酬改革: 协助CEO对整个公司人员职级、薪酬、考核、绩效等进行深入的改革，建立人才的评价体系与晋升体系，从而提升员工的人效水平，改善公司的经营指标；\\n5、数据系统化搭建: 协同技术部门已经业务部门需求，重新对整个公司的消耗、充值、回款、成本、商务政策等进行梳理以及取数来源、时间点、媒体等综合优化评估，形成新的一套更为科学准确的核算逻辑，提升公司管理管理决策的数据依据以及公司正规化、体系化建设；\\n6、总经办助理: 协助董事长、CEO制定上述相关目标以及关键行动方案，并协同相关部门落实。\\n二、运营管理\\n1、对运营团队根据行业、人效、产出、客户赛道等进行组织架构的调整和划分，确保人尽其职，产出最大化；\\n2、结合技术部门前沿发展，推动在运营部门大力使用智投、创意畅享平台比较先进的工作，从而提升单个运营的大基建能力；\\n3、以单个运营团队作为战斗单元，结合客户标签，完善公司客户分配机制，确保单个客户消耗产出最大化。\\n4、案列分析: 组织人员对优秀的素材、数据等进行分析，提炼相关案列以及成功关键要素，在运营内部和销售部门进行推广。\\n5、运营专业化信息建设: 对公司整个运营团队通过广告投放数据、行业数据等协助ceo建立大数据中心，对公司在职运营、客户、行业等完成标签化，分类等工作，实现运营体系化、数据高效化、投放智能化。\\n三、客户和风险管理\\n1、优化销售流程: 对客户引入、商务政策、授信、合同等全流程摸底，从效率、风险、信息化建设的角度，完善该条流程；\\n2、新客户建立评估机制: 对销售端客户引入进行评估，并制定相关的标准，确保客户质量；\\n3、老客户

JSONDecodeError: Expecting value: line 1 column 2 (char 1)

In [13]:
x = ['{"companyName": "深圳市亿科数字科技有限公司", "description": "工作职责:\\n一、公司管理\\n1、确定发展方向: 结合公司特点、竞争优势、行业机会等信息，确定公司未来主要从事的行业和赛道，如移动互联网、创意为主、技术导向三大战略；\\n2、企业文化建设: 和ceo一道，深入对公司文化进行总结、分析，通过投票、讨论、调研等方式，最终确定公司文化，并在各个事业部进行推行落实；\\n3、内部流程优化: 协同销售、运营、创意、财务、技术、商务等部门对整个公司内部的流程进行梳理，最终落脚为公司信息化系统建设，从而提升公司运转的效率和规避相关的风险；\\n4、薪酬改革: 协助CEO对整个公司人员职级、薪酬、考核、绩效等进行深入的改革，建立人才的评价体系与晋升体系，从而提升员工的人效水平，改善公司的经营指标；\\n5、数据系统化搭建: 协同技术部门已经业务部门需求，重新对整个公司的消耗、充值、回款、成本、商务政策等进行梳理以及取数来源、时间点、媒体等综合优化评估，形成新的一套更为科学准确的核算逻辑，提升公司管理管理决策的数据依据以及公司正规化、体系化建设；\\n6、总经办助理: 协助董事长、CEO制定上述相关目标以及关键行动方案，并协同相关部门落实。\\n二、运营管理\\n1、对运营团队根据行业、人效、产出、客户赛道等进行组织架构的调整和划分，确保人尽其职，产出最大化；\\n2、结合技术部门前沿发展，推动在运营部门大力使用智投、创意畅享平台比较先进的工作，从而提升单个运营的大基建能力；\\n3、以单个运营团队作为战斗单元，结合客户标签，完善公司客户分配机制，确保单个客户消耗产出最大化。\\n4、案列分析: 组织人员对优秀的素材、数据等进行分析，提炼相关案列以及成功关键要素，在运营内部和销售部门进行推广。\\n5、运营专业化信息建设: 对公司整个运营团队通过广告投放数据、行业数据等协助ceo建立大数据中心，对公司在职运营、客户、行业等完成标签化，分类等工作，实现运营体系化、数据高效化、投放智能化。\\n三、客户和风险管理\\n1、优化销售流程: 对客户引入、商务政策、授信、合同等全流程摸底，从效率、风险、信息化建设的角度，完善该条流程；\\n2、新客户建立评估机制: 对销售端客户引入进行评估，并制定相关的标准，确保客户质量；\\n3、老客户进行分级管理: 对于已合作的客户从消耗、回款、毛利、经营利润等角度，进行评级，确保公司资源合理使用；\\n4、大客户管理: 完善公司大客户管理办法，针对销售行为进行约束，如拜访、招标、关系深度等进行明确，并推动成立公司大客户部门；\\n5、客户档案信息化: 针对大客户的特殊性，把信息化作为承载，确保大客户在自从合作以来，每一个行为，每一次的合作，都有档案可循。\\n6、客户分配合理化: 考虑到广告行业客户的流动性，针对已经合作的客户，制定客户的淘汰要求和标准，确保运力得到最大使用，并减少销售和运营跨部门的冲突。\\n7、客户风险控制: 制定公司逾期管理办法，根据客户分类、风险等级等明确销售、法务、财务等在应收款阶段各自的职责和主要责任，确保逾期风险得以控制；\\n4、 法务管理\\n1、合同管理: 结合业务情况，制定公司业务合作合同流程、模板、审核要点、审核节点、预警机制等；\\n2、起诉管理: 针对不同类别的逾期、员工违规、客户应收、应诉等进行评估，并制定相关的规则，尽最大化保证公司利益。\\n3、外部律所: 制定相关律所引进的标准、价格管理、举证责任等进行规范，确保相关案子能顺利进行。\\n5、 销售支持\\n1、拆分销售职能: 对销售岗位进行关键内容拆分，让销售更聚焦在客户开发、回款、关系维护，消耗提升上面；\\n2、组建新部门: 全面对非销售工作进行支撑，如结算、充值、开户、邮件等处理；\\n3、明确职责和定位: 职能职责、部门导向等，培养独挡一面的销售支持部负责人。\\n工作业绩:\\n1、协助ceo完成公司薪酬改革，特别是一线的销售、运营，创意等岗位，对整个公司所有员工进行定级定岗，推动公司全面导入kpi管理；\\n2、明确公司使命愿景目标，并在具体工作中，同事业部一起推进，确保企业文化和具体工作落实在一起；\\n3、客户系统化管理，拉通前端后端，确保从客户引入到上线，更为高效；\\n4、成立销售支持、运力调度室、大客户部，更好的支持公司相关业务增长120%。", "endDate": "2021-12", "id": "2c9207157dbedf8d017dc28fbc5d3366", "jobTitle": "副总裁/副总经理", "startDate": "2020-02"}', '{"companyName": "深圳市蜂联科技有限公司（奇虎360）", "description": "工作职责:\\n1、 策略制定\\n1、 结合公司年度发展战略，制定电商部门相关战略，并对战略进行拆解，保证公司战略的延续，部门战略的执行；\\n2、 结合市场和公司目标，制定公司的销售模型以及盈利模型，虽然这几年变化比较大，基本上围绕利润和销量在做平衡发展；\\n2、 渠道管理\\n1、基于对行业和竞争的充分理解和分析，制定和修正最近3-5年的渠道策略；\\n2、根据公司年度销售额和重点单品销量目标，拆分各个渠道销售任务，并结合各个渠道的差异性，实时做一些调整；\\n3、根据产品、渠道、用户的特点，布局电商渠道，做到优势渠道的发掘、重点渠道的提升；\\n4、从渠道的发展以及渠道的布局角度，制定渠道的管理制定和考核考核，激励、提升、发展重点渠道；\\n5、改变单一的奖励制度，对渠道进行电商、产品、营销的培训，授之以鱼，不如授之以渔。\\n三、产品管理\\n1、结合公司产品路线图，建议和调整电商部门产品规划，以保证产品更接近市场，更符合公司需求；\\n2、在新产品立项阶段，利用电商数据的准确性和多样性，为新产品的定义提供用户需求分析；\\n3、新建和优化产品上市流程，采取宣传、试售、收集分析、宣传、上市、推广等组合手段，确保新品上市成功！\\n4、在产品进入售卖阶段后，重新定义产品买点，并定期优化和调整产品的买点，确保更贴近市场需求；\\n5、定价管理，根据市场竞争情况以及公司成本、毛利要求，对产品进行定价，并根据产品的迭代和生命周期，对价格进行调整；\\n四、营销策划\\n1、结合公司市场、品牌部门以及电商部门的特点，制定该年度部门的营销策略；\\n2、制定部门年度营销投入预算和执行计划，确保投入产出的最大化；\\n3、建立部门的评审机制和讨论机制，对营销相关的点，进行充分的讨论，最终保证风险降到最低；\\n4、开发新的宣传和推广渠道，尝试去电商化的宣传，在新媒体、app联盟、内容联盟、视频网站等方面进行尝试；\\n5、制定年度重要营销方案，包括新品首发、大型促销、自建主题的促销case，做到行业投产比最好，传播效果佳的目标；\\n6、对电商部门的用户通过微信、微淘、公众号的管理，分别从二次购买、促销传播、新品推广、安装售后等方面进行利用和管理；\\n7、梳理行业、竞争、自我产品、流量、转化等数据，寻找到新的机会和突破点，特别是针对自营平台，京东和天猫旗舰店，结合数据变化，随时做调整；\\n8、制定和审核爆款打造计划，并通过日会、评审、讨论、分析等手段保证爆款 打造目标的完成。\\n五、部门管理\\n1、结合公司要求以及业务发展情况，制定、修改、调整部门的组织架构，形成三大业务主体与一个后端业务支撑体系；\\n2、制定、优化和调整人员的考核方案，做到全员有价值，全员有考核，保证团队活力和积极性；\\n3、总结部门价值观，在自驱力、赢、业务结果目标导向三个方面进行宣导；\\n4、固化部门管理标准模版，形成固定的日报、周报、月报以及会议，建立顺畅的监督和反馈机制；\\n六、经营管理\\n1、推行部门预算机制，制定和审核每个月的财务预算，次月月初再次确定预算的执行情况，并对下个月预算作出适当的调整；\\n2、调整经销商的合作方式、支付方式、回款周期等，确保公司风险进一步在业务体系就得到降低；\\n3、协同公司采购、商务，在部门采购以及合作供应商上面进行选择，并定期对成本进行review，确保竞争优势；\\n4、按照季度和年度为单位，对部门所有的费用进行分析、总结，重点对产品成本、营销费用两个大头进行管理。\\n工作业绩:\\n1、销售业绩: 公司业绩从最开始的不到1个亿到截止到2018年的12个亿，2019年15个亿的目标；\\n2、爆品打造: 先后在京东、天猫平台将5个单品， 做到了同类目前三，并维持了至少6个月以上的时间；\\n3、渠道结构: 改变了以前单一的b2c渠道，增加了代理、旗舰店渠道，并逐渐形成了三分天下的趋势，特别是天猫旗舰店，开店一年，就做到类目前三店铺，月销过3千万；\\n4、人员培养: 这几年的时间，培养了三位销售主管，一名策划主管，并都做到了独挡一面，团队的承接机制得到了保存；\\n5、团队建设: 铁血的手段和强势的管理，培养了一支能带打战，能打硬仗的团队；\\n6、毛利贡献: 整个部门销售占据公司60-70%的销售收入，贡献公司50-60%的毛利！\\n7、风险管理: 这几年做了将近60个亿的销售，唯一的风险就是将近30万的货款，合作风险、内控风险、税务风险都没有发生。", "endDate": "2019-09", "id": "2c9207157dbedf8d017dc28fbc5d3367", "jobTitle": "电子商务部总监", "startDate": "2013-08"}', '{"companyName": "深圳市吉祥腾达科技有限公司", "description": "工作职责:\\n1、 部门管理\\n1、结合公司实际情况，制定部门的使命、愿景、目标和战略；\\n2、搭建部门工作流程，协助人资制定部门的考核；\\n3、建立日报和周会制定，完善对团队成员的有效监督和指导；\\n4、协调公司产品、营销、供应链以及售后资源，助力销售工作的开展。\\n2、 业绩达成\\n1、按照季节指数，按照月度、产品线、客户、分公司维度分解销售任务；\\n2、带领销售团队，按照年度、季度、月度销售计划，完成公司销售目标；\\n3、指导下属对京东、淘宝、天猫、易讯、1号店等平台销售工作的开展 。\\n3、 渠道构建\\n1、依据产品线和区域特征，重点布局天猫代理核心客户以及普通客户；\\n2、分析国内电商平台特点，分A\\\\B\\\\C类渠道进行工作的开展以及监督；\\n3、制定代理商合作标准，配合分公司、客户经理对于重点渠道的开拓；\\n4、根据产品型号，组建产品推广的战略同盟，平衡合作伙伴利益关系。\\n4、 市场营销\\n1、以竞争为导向，制定价格策略，并平衡电商与传统之间的差异；\\n2、制定年度的产品和促销推广计划，提升各大平台产品和品牌曝光；\\n3、分析竞争对手，优化产品图片、卖点、介绍等提升产品的转换率；\\n4、调研消费者需求，利用MM流程以及IPD，定义符合电商的产品；\\n5、通过优化关键词、产品命名、网站结构分析等，提升店铺流量；\\n6、整合淘宝、京东以及第三方的营销资源，组织大型的营销活动；\\n7、搜集淘宝终端消费者信息，建立CRM系统，加强会员管理和利用。\\n5、 团队建设\\n1、以周为单位，每周开展市场营销、渠道相关书籍的学习以及交流；\\n2、培养全员的竞争意识以及战斗精神，每周周会开展批评和自我批评；\\n3、按照公司内部三级管理者标准，重点培养华东和华北的平台经理。\\n工作业绩:\\n1、实现公司电子商务月均销售从200多万到2011年的月均3000多万，2012年月均5000万；\\n2、在天猫商城上先后培养了5款类目第一的产品；\\n3、京东商城相关类目上腾达产品最高峰达到了8款；\\n4、1号店、苏宁易购、国美商城腾达销量类目第一；\\n5、培养了一个技术支持主管，华东和华北平台经理各一名。", "endDate": "2013-05", "id": "2c9207157dbedf8d017dc28fbc5d3368", "jobTitle": "电子商务部经理", "startDate": "2010-05"}', '{"companyName": "康佳集团股份有限公司", "description": "工作职责:\\n1、分解年度、季度、月度销售目标，带领客户经理完成销售计划；\\n2、负责公司整体产品在京东、淘宝、易讯、卓越等网站的整体操盘；\\n3、规划各个网站不同阶段促销，并审核专场促销活动创意；\\n4、搭建公司电子商务组织以及梳理公司内部流程；\\n5、分析竞品，推行差异化的产品策略以及优化产品结构。", "endDate": "2010-05", "id": "2c9207157dbedf8d017dc28fbc5d3369", "jobTitle": "直销经理", "startDate": "2008-01"}']

In [21]:
json.loads(str(x[0]))

{'companyName': '深圳市亿科数字科技有限公司',
 'description': '工作职责:\n一、公司管理\n1、确定发展方向: 结合公司特点、竞争优势、行业机会等信息，确定公司未来主要从事的行业和赛道，如移动互联网、创意为主、技术导向三大战略；\n2、企业文化建设: 和ceo一道，深入对公司文化进行总结、分析，通过投票、讨论、调研等方式，最终确定公司文化，并在各个事业部进行推行落实；\n3、内部流程优化: 协同销售、运营、创意、财务、技术、商务等部门对整个公司内部的流程进行梳理，最终落脚为公司信息化系统建设，从而提升公司运转的效率和规避相关的风险；\n4、薪酬改革: 协助CEO对整个公司人员职级、薪酬、考核、绩效等进行深入的改革，建立人才的评价体系与晋升体系，从而提升员工的人效水平，改善公司的经营指标；\n5、数据系统化搭建: 协同技术部门已经业务部门需求，重新对整个公司的消耗、充值、回款、成本、商务政策等进行梳理以及取数来源、时间点、媒体等综合优化评估，形成新的一套更为科学准确的核算逻辑，提升公司管理管理决策的数据依据以及公司正规化、体系化建设；\n6、总经办助理: 协助董事长、CEO制定上述相关目标以及关键行动方案，并协同相关部门落实。\n二、运营管理\n1、对运营团队根据行业、人效、产出、客户赛道等进行组织架构的调整和划分，确保人尽其职，产出最大化；\n2、结合技术部门前沿发展，推动在运营部门大力使用智投、创意畅享平台比较先进的工作，从而提升单个运营的大基建能力；\n3、以单个运营团队作为战斗单元，结合客户标签，完善公司客户分配机制，确保单个客户消耗产出最大化。\n4、案列分析: 组织人员对优秀的素材、数据等进行分析，提炼相关案列以及成功关键要素，在运营内部和销售部门进行推广。\n5、运营专业化信息建设: 对公司整个运营团队通过广告投放数据、行业数据等协助ceo建立大数据中心，对公司在职运营、客户、行业等完成标签化，分类等工作，实现运营体系化、数据高效化、投放智能化。\n三、客户和风险管理\n1、优化销售流程: 对客户引入、商务政策、授信、合同等全流程摸底，从效率、风险、信息化建设的角度，完善该条流程；\n2、新客户建立评估机制: 对销售端客户引入进行评估，并制定相关的标准，确保客户质量；\n3、老客户进行分级管理: 对于已合作的客户从消

In [16]:
# 显示所有的列
all_columns = list(all_data.columns)
all_data[all_columns[79:]]

,version,workYear,avatar,eduTracks,jobTracks,projectTracks,ds.1
0,1,13.0,NaN,"['{""degree"": ""本科"", ""endDate"": ""2008-07"", ""id"":...","['{""companyName"": ""深圳市亿科数字科技有限公司"", ""descriptio...","['{""companyName"": ""*深圳市蜂联科技有限公司（奇虎360）"", ""desc...",20220821
1,1,7.0,NaN,"['{""degree"": ""本科"", ""endDate"": ""2014-06"", ""id"":...","['{""companyName"": ""北京美餐好客科技有限公司"", ""companyNatu...","['{""description"": ""项目描述: 母婴类人群，包含童装、童鞋、奶粉。\\n运...",20220821
2,1,14.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2012-09"", ""id"":...","['{""companyName"": ""Continental Tires (China) L...",NaN,20220821
3,1,3.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2019-06"", ""id"":...","['{""companyName"": ""北京星制科技有限公司（全民快乐）"", ""descrip...","['{""description"": ""主播表现，直播平均 2 场，平台新用户 12926，主...",20220821
4,1,5.0,https://image0.lietou-static.com/big/5f8fa3bde...,"['{""degree"": ""硕士"", ""endDate"": ""2016-07"", ""id"":...","['{""companyName"": ""酷狗音乐"", ""companyTags"": ""互联网+...","['{""companyName"": ""酷狗音乐"", ""description"": ""酷狗20...",20220821
...,...,...,...,...,...,...,...
81291,1,1.0,NaN,"['{""endDate"": ""2022-07"", ""unified"": false, ""de...","['{""endDate"": ""2022-07"", ""companyName"": ""江门市新会...","['{""name"": null}']",20220821
81292,1,23.0,NaN,"['{""schoolTag"": ""211"", ""endDate"": ""1999-07"", ""...","['{""endDate"": ""2022-07"", ""companyName"": ""上海微创医...","['{""name"": null}']",20220821
81293,1,23.0,NaN,"['{""schoolTag"": ""211"", ""endDate"": ""1999-07"", ""...","['{""endDate"": ""2022-07"", ""companyName"": ""上海微创医...","['{""name"": null}']",20220821
81294,1,5.0,NaN,"['{""endDate"": ""2015-06"", ""unified"": false, ""de...","['{""endDate"": ""2020-07"", ""companyName"": ""金山云"",...","['{""name"": null}']",20220821


In [4]:
# 注意一下，这里的列名和sample pipeline中的列名有一点大小写区别，问题不大
cv_columns = ['cv_id', 'currentPosition', 'desiredPosition', 'industry', 'desiredIndustry', 'majorName', 'skills', 'eduTracks', 'jobTracks', 'projectTracks']
jd_columns = ['jd_id', 'title', 'category_name', 'tags', 'description', 'requirement']

对于上述文本字段的一些看法:
* industry和desiredIndustry的缺失值比较严重
* majorName感觉字段错位，里面有很多学校信息.建议重新入库。 

In [17]:
jd_data = all_data[jd_columns]
jd_data

,jd_id,title,category_name,tags,description,requirement
0,2c9207157bcbb76b017bcde2b46d04cd,市场营销负责人,市场营销,"[""营销"",""管理"",""销售"",""战略规划"",""管理工作""]",1、岗位职责:2、主导制定公司营销战略规划并参与公司整体战略规划制定。3、全面管理公司的市场...,1、电商2、线下营销
1,2c9207157bcbb76b017bf7f2bbbb3153,会员管理经理,用户运营,"[""会员运营"",""会员管理"",""管理"",""营销"",""社群运营""]",1、岗位职责：2、1. 负责“九木杂物社”品牌全渠道会员体系的管理，规划品牌会员运营方向，包...,1、有会员管理、会员活动经验
2,2c9207157bf94d99017c108f769405e0,产品市场经理,产品经理,"[""EXCEL"",""OFFICE"",""PPT"",""UI""]",1、•Support regional strategic planning and mar...,1、find jd
3,2c9207157d2f0ec5017d5a9265ba33d9,海外区域运营经理/高级经理,运营,"[""管理"",""运营"",""团队管理"",""电商"",""游戏""]",1、负责短视频业务的运营策略制定，配合完成公司的营收业务指标；2、主导团队的业务方向，包括但...,1、本科及以上学历优先，3年以上社交、直播、游戏、电商或相关行业运营工作经验；2、英语听说读...
4,2c9207157d631abb017d6eed1c79139f,高级项目经理,高级管理,"[""项目管理"",""推广"",""归档"",""风险评估"",""交货""]",1、负责从产品建立到第一批生产交付的全过程项目管理;2、 负责实现产品要求、进度、成本、质量...,1、 本科及以上学历，计算机科学或相关专业。能用英语交流者优先;2、5年以上可穿戴相关产品项...
...,...,...,...,...,...,...
81291,8a69f6f8823a93b901823d891b761d51,技术经理,机械设备工程师,"[""材料"",""考核"",""维修"",""技术标准"",""不锈钢""]",职责描述：1、全面负责公司技术标准制、各岗位的工艺制定跟改善；2、重要设备参数收集管控包各棍...,任职要求：1、具备压延400系不锈钢材料技术相关工作经验，能对水扎四六棍扎异常原因分析解决、...
81292,8a69c468826ee9b40182831bedfc5e79,中央市场总监,市场总监,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...
81293,8a69f6f8823a93b9018247c30aca4e3d,中央市场总监,市场/营销,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...
81294,2c92071580dd3ab70180f18714e119c0,java开发工程师（不限方向）,Java,"[""java"",""微服务/性能优化"",""SQL"",""架构"",""需求分析""]",1、负责公司OA产品的功能规划、需求分析设计、技术实现和用户体验；2、负责所属模块的代码开发...,1、本科及以上学历，2年以上的Java开发及应用经验，计算机或者相关专业;2、良好的问题理解...


In [25]:
cv_data = all_data[cv_columns]
cv_data

,cv_id,currentPosition,desiredPosition,industry,desiredIndustry,majorName,skills,eduTracks,jobTracks,projectTracks
0,2c9207157bf91042017c0d2beef00470,副总裁/副总经理,NaN,NaN,NaN,国际经济与贸易,"[""Ipd"",""App"",""Kpi"",""Crm"",""View"",""优化"",""策划"",""营销""...","['{""degree"": ""本科"", ""endDate"": ""2008-07"", ""id"":...","['{""companyName"": ""深圳市亿科数字科技有限公司"", ""descriptio...","['{""companyName"": ""*深圳市蜂联科技有限公司（奇虎360）"", ""desc..."
1,2c9207157bf91042017c0d979cbb047d,数字化营销经理,NaN,餐饮业,NaN,电气工程及其自动化,"[""H5"",""C1"",""Seo"",""Sem"",""Top"",""Kpi"",""Crm"",""优化"",...","['{""degree"": ""本科"", ""endDate"": ""2014-06"", ""id"":...","['{""companyName"": ""北京美餐好客科技有限公司"", ""companyNatu...","['{""description"": ""项目描述: 母婴类人群，包含童装、童鞋、奶粉。\\n运..."
2,2c9207157bf91042017c7232fbab3be1,NaN,NaN,NaN,NaN,Marketing,"[""Based"",""Control"",""Sap Crm"",""Improve"",""Indivi...","['{""degree"": ""硕士"", ""endDate"": ""2012-09"", ""id"":...","['{""companyName"": ""Continental Tires (China) L...",NaN
3,2c9207157d308592017d4a94cb1113a1,泰国国家经理,NaN,NaN,NaN,企业管理,"[""App"",""Resume"",""运营"",""策划"",""营销"",""搭建"",""笔译"",""舞蹈"",...","['{""degree"": ""硕士"", ""endDate"": ""2019-06"", ""id"":...","['{""companyName"": ""北京星制科技有限公司（全民快乐）"", ""descrip...","['{""description"": ""主播表现，直播平均 2 场，平台新用户 12926，主..."
4,2c9207157dabb0a3017dbca16cdf4c26,项目经理,项目经理/主管,互联网+,"智能硬件,互联网+",英语,"[""智能硬件""]","['{""degree"": ""硕士"", ""endDate"": ""2016-07"", ""id"":...","['{""companyName"": ""酷狗音乐"", ""companyTags"": ""互联网+...","['{""companyName"": ""酷狗音乐"", ""description"": ""酷狗20..."
...,...,...,...,...,...,...,...,...,...,...
81291,8a69d7c28219945301823d8a08d14e70,生产工、包装工,生产制造/机械设计/制造/机械设备工程师,NaN,NaN,无,电气,"['{""endDate"": ""2022-07"", ""unified"": false, ""de...","['{""endDate"": ""2022-07"", ""companyName"": ""江门市新会...","['{""name"": null}']"
81292,8a69d7c282199453018247bac4a36859,高级市场总监,市场/市场/营销/市场营销,NaN,NaN,上海大学,"搭建,翻译,功能,医学,数据库,Education,产品管理,产品经理,医疗器械,英语听说读写","['{""schoolTag"": ""211"", ""endDate"": ""1999-07"", ""...","['{""endDate"": ""2022-07"", ""companyName"": ""上海微创医...","['{""name"": null}']"
81293,8a69d7c282199453018247bac4a36859,高级市场总监,市场/市场/营销/市场营销,NaN,NaN,上海大学,"搭建,翻译,功能,医学,数据库,Education,产品管理,产品经理,医疗器械,英语听说读写","['{""schoolTag"": ""211"", ""endDate"": ""1999-07"", ""...","['{""endDate"": ""2022-07"", ""companyName"": ""上海微创医...","['{""name"": null}']"
81294,8a69d7db826df6c201827b4338cd60c2,java,技术/后端开发/Java,NaN,NaN,太原理工大学,"Bi,Cdn,Elk,Git,Yii,App,Php,Ext,Etl,Java,Sina,H...","['{""endDate"": ""2015-06"", ""unified"": false, ""de...","['{""endDate"": ""2020-07"", ""companyName"": ""金山云"",...","['{""name"": null}']"


In [19]:
cv_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81296 entries, 0 to 81295
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   cv_id            81296 non-null  object
 1   currentPosition  78952 non-null  object
 2   desiredPosition  51408 non-null  object
 3   industry         15671 non-null  object
 4   desiredIndustry  9345 non-null   object
 5   majorName        80395 non-null  object
 6   skills           74527 non-null  object
 7   eduTracks        81293 non-null  object
 8   jobTracks        80324 non-null  object
 9   projectTracks    55283 non-null  object
dtypes: object(10)
memory usage: 6.2+ MB


In [23]:
with open('./majorname.txt', 'w') as fo:
    for major in cv_data['majorName']:
        fo.write("{}\n".format(major))


jd可以做3个向量
* title + category_name + tags
* description
* requirement

cv可以做4个向量：
* currentPosition + desiredPosition
* skills
* jobTracks
* projectTracks

向量1：title + category_name + tags

In [ ]:
# step1：缺失值填充

jd_data['title'].fillna('', inplace=True)
jd_data['category_name'].fillna('', inplace=True)
jd_data['tags'].fillna('[]', inplace=True)

In [ ]:
# step2：定义新列，将title + category_name合并

col_name1 = 'title'
col_name2 = 'category_name'

def col_merge_str_fun(series):
    '''
    适用于文本列合并
    '''
    return series[col_name1] + ' ' + series[col_name2]

jd_data['title_category'] = jd_data.apply(col_merge_str_fun, axis=1)
jd_data

In [ ]:
# step3：将新列title_category通过jieba分词处理成列表

col_name = 'title_category'

def col_jieba_fun(series):
    col = series[col_name]

    # 字符串变列表
    if col.startswith("[") and col.endswith("]"):
        col = json.loads(col)
    else:
        col = re.split(",|，|/| ", col)

    # 对于中文，进入jieba前不需要添加空格
    # 不过，如果是中英文混合，就必须空格了
    col_str = " ".join(col)
    col_list = jieba.lcut(col_str, cut_all=False)
    return col_list

jd_data['title_category_jieba'] = jd_data.apply(col_jieba_fun, axis=1)
jd_data


In [ ]:
# step4：将tags列通过jieba分词处理成列表

col_name = 'tags'

def col_jieba_fun(series):
    col = series[col_name]

    # 字符串变列表
    if col.startswith("[") and col.endswith("]"):
        col = json.loads(col)
    else:
        col = re.split(",|，|/| ", col)

    # 对于中文，进入jieba前不需要添加空格
    # 不过，如果是中英文混合，就必须空格了
    col_str = " ".join(col)
    col_list = jieba.lcut(col_str, cut_all=False)
    return col_list

jd_data['tags_jieba'] = jd_data.apply(col_jieba_fun, axis=1)
jd_data


In [ ]:
# step5：将title_category_jieba和tags_jieba两个列表合并

col_name1 = 'title_category_jieba'
col_name2 = 'tags_jieba'

def col_merge_list_fun(series):
    '''
    适用于列表列合并
    '''
    return series[col_name1] + series[col_name2]

jd_data['title_category_tags_jieba'] = jd_data.apply(col_merge_list_fun, axis=1)
jd_data


In [ ]:
# step6：过滤title_category_tags_jieba, 得到以空格分割的字符串

col_name = 'title_category_tags_jieba'
def col_jieba_filter_fun(series):

    col_list_filter = []
    # 得到tag列表
    col_list = series[col_name]
    # print(tags_list[0])

    pun_masks_english = [",", ".", "/", "[", "]", "{", "}", "(", ")", ":", "*", "#", "!", " ", "\"", "\\"]
    pun_masks_chinese = ["，", "。", "、", "（", "）", "：", "！", "”", "“"]
    pun_masks = pun_masks_english + pun_masks_chinese

    # 过滤
    for tag in col_list:
        # 转中文简体
        tag = convert(tag, "zh-hans")
        # 转英文小写
        tag = tag.lower()

        # 过滤数字
        if tag.isdigit():
            continue
        
        # 过滤单个字符
        if len(tag) <= 1:
            continue
        
        # 过滤标点
        flag = 1
        for pun in pun_masks:
            if pun in tag:
                flag = 0
                break
        if flag == 1:
            col_list_filter.append(tag)
    return " ".join(col_list_filter)

jd_data['title_category_tags_jieba_filter'] = jd_data.apply(col_jieba_filter_fun, axis=1)
jd_data

In [ ]:
# step7：将title_category_tags_jieba_filter转成tfidf向量

def get_tfidf(df, col_name):
    # col_name必须是以空格分割的字符串
    text = df[col_name]
    
    vectorizer = TfidfVectorizer()
    # 返回的是nparray
    vector = vectorizer.fit_transform(text)
    return pd.DataFrame(vector.toarray()), vectorizer

tfidf, vectorizer = get_tfidf(jd_data, 'title_category_tags_jieba_filter')
tfidf


In [ ]:
vectorizer.vocabulary_

In [ ]:
# step8：将tfidf降维到n维

def get_tfidf_pca(tfidf, n=20):
    pca = PCA(n_components=n)
    tfidf_pca = pca.fit_transform(tfidf)
    tfidf_pca = pd.DataFrame(tfidf_pca)
    return tfidf_pca

tfidf_pca = get_tfidf_pca(tfidf, 10)
tfidf_pca

向量2：description

In [ ]:
# step1：缺失值填充

jd_data['description'].fillna('', inplace=True)

In [ ]:
# step2：将desciption列通过jieba分词处理成列表

col_name = 'description'

def col_jieba_fun(series):
    col = series[col_name]

    # 字符串变列表
    if col.startswith("[") and col.endswith("]"):
        col = json.loads(col)
    else:
        col = re.split(",|，|/| ", col)

    # 对于中文，进入jieba前不需要添加空格
    # 不过，如果是中英文混合，就必须空格了
    col_str = " ".join(col)
    col_list = jieba.lcut(col_str, cut_all=False)
    return col_list

jd_data['description_jieba'] = jd_data.apply(col_jieba_fun, axis=1)
jd_data


In [ ]:
# step3：过滤description_jieba，得到以空格分割的字符串

col_name = 'description_jieba'

def col_jieba_filter_fun(series):

    col_list_filter = []
    # 得到tag列表
    col_list = series[col_name]
    # print(tags_list[0])

    pun_masks_english = [",", ".", "/", "[", "]", "{", "}", "(", ")", ":", "*", "#", "!", " ", "\"", "\\"]
    pun_masks_chinese = ["，", "。", "、", "（", "）", "：", "！", "”", "“"]
    pun_masks = pun_masks_english + pun_masks_chinese

    # 过滤
    for tag in col_list:
        # 转中文简体
        tag = convert(tag, "zh-hans")
        # 转英文小写
        tag = tag.lower()

        # 过滤数字
        if tag.isdigit():
            continue
        
        # 过滤单个字符
        if len(tag) <= 1:
            continue
        
        # 过滤标点
        flag = 1
        for pun in pun_masks:
            if pun in tag:
                flag = 0
                break
        if flag == 1:
            col_list_filter.append(tag)
    return " ".join(col_list_filter)

jd_data['description_jieba_filter'] = jd_data.apply(col_jieba_filter_fun, axis=1)
jd_data

In [ ]:
# step4：将description_jieba_filter转成tfidf向量

def get_tfidf(df, col_name):
    # col_name必须是以空格分割的字符串
    text = df[col_name]
    
    vectorizer = TfidfVectorizer()
    # 返回的是nparray
    vector = vectorizer.fit_transform(text)
    return pd.DataFrame(vector.toarray()), vectorizer

tfidf, vectorizer = get_tfidf(jd_data, 'description_jieba_filter')
tfidf

In [ ]:
vectorizer.vocabulary_

In [ ]:
# step5：将tfidf降维到n维

def get_tfidf_pca(tfidf, n=20):
    pca = PCA(n_components=n)
    tfidf_pca = pca.fit_transform(tfidf)
    tfidf_pca = pd.DataFrame(tfidf_pca)
    return tfidf_pca

tfidf_pca = get_tfidf_pca(tfidf, 10)
tfidf_pca

In [36]:
import pandas as pd

dict1 = {"流浪地球": [25,45,65,89],
        "唐人街探案":(89,56,89,17),
        "谋杀2":[19,46,49,79]}

index = ["第{}天".format(i) for i in range(1,5)]
df = pd.DataFrame(dict1, index)

df

,流浪地球,唐人街探案,谋杀2
第1天,25,89,19
第2天,45,56,46
第3天,65,89,49
第4天,89,17,79


In [47]:
def fun(series, col1, col2):
    return str(series[col1]) + str(series[col2])

df['new_col'] = df.apply(fun, axis=1, args=('唐人街探案', '谋杀2', ))
df

,流浪地球,唐人街探案,谋杀2,new_col
第1天,25,89,19,8919
第2天,45,56,46,5646
第3天,65,89,49,8949
第4天,89,17,79,1779
